<img width = 130 height = 130 align = left src="tfkeras.jpg">
 
# Tensorflow for Deep Learning 

Learning Objectives: *By the end of this assignment, you should be comfortable with using Keras Sequential and Functional APIs for constructing deep learning models. You should be comfortable with debugging common modeling errors and researching Tensorflow documentation for various open-ended tasks.*

**Keras** is a deep learning API that runs on top of Tensorflow, with **layers** and **models** as the core data structures. In Tensorflow 2.0, modeling functionalities are organized under the Keras namespace. (Optional: read about v1 --> v2 API cleanup [here](https://github.com/tensorflow/community/blob/master/rfcs/20180827-api-names.md))

Keras provides a clean, approachable interface with abstractions and building blocks for easy prototyping and modeling customizations. 

In [4]:
# tensorflow_version works only in colab
try: 
    %tensorflow_version 2.x
except Exception: 
    pass

import tensorflow as tf
tf.__version__

'2.3.1'

In [5]:
# several examples use the mnist dataset -- hence import & split into train/valid/test sets

"""
Background on Fashion-MNIST Dataset: 
Fashion-MNIST is a dataset of Zolando's article images, containing 60k training samples and 10k test samples.  
Each 28x28 greyscale image belongs to one of ten classes (t-shirt/top, trouser, pullover, dress, coat, sandal, 
shirt, sneaker, bag, ankle boot). Each pixel-value is an integer between 0 and 255, where higher means darker.
"""
from tensorflow.keras.datasets import mnist
(x_train_full, y_train_full), (x_test,  y_test) = mnist.load_data()
x_train_full = x_train_full.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

# amount of images to be in the validation set (80/20 split)
split_amt = int(len(x_train_full) * .2)

x_train, x_valid = x_train_full[split_amt:] / 255.0, x_train_full[:split_amt] / 255.0
y_train, y_valid = y_train_full[split_amt:], y_train_full[:split_amt]

## 1. Sequential API - Basics

The Sequential API allows one to construct the simplest type of model: one with a linear stock of layers -- ie. layers created in a step by step fashion. 

In the example below, we are interested in constructing a model for 10-class classification with the Fashion-MNIST dataset. Carefully examine the code and associated comments below.  

In [ ]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# creates a list of layer definitions
seq_model = Sequential([ 
    # generate 16 3x3 filters & relu activation 
    Conv2D(16, (3, 3), activation="relu", input_shape=(28, 28, 1)), 
    # 2x2 max-pooling -- every four pixels, the largest value survives
    MaxPooling2D(2, 2), 
    # generate 32 3x3 filters & relu activation 
    Conv2D(32, (3, 3), activation="relu"), 
    # 2x2 max-pooling -- every four pixels, the largest value survives
    MaxPooling2D(2, 2), 
    # flattens to 1D array 
    Flatten(), 
    # fully connected hidden layer with 256 neurons & relu activation
    Dense(256, activation="relu"), 
    # fully connected hidden layer with 128 neurons & relu activation 
    Dense(128, activation="relu"), 
    # fully connected output layer with 10 neurons & softmax activation 
    Dense(10, activation="softmax")
])

# displays model layers (+ layer (type), output shape, param #)
seq_model.summary()

The next step following model instantiation is to call the *compile()* method and specify a loss function, optimizer, and metrics.

In [ ]:
                  # loss -- String (name of objective function), objective function, or Loss instance.
seq_model.compile(loss = "sparse_categorical_crossentropy",
                  # optimizer -- String (name of optimizer) or optimizer instance. 
                  optimizer = "sgd", 
                  # list of metrics to be evaluated by the model during training and testing.
                  # each can be a String (name of built-in function), function, or Metric instance. 
                  metrics = ["accuracy"])

The next step following compiling is to train the model by calling the *fit()* method. <br> Doing so returns a *History* object, with its *History.history* attribute holding records of training loss and metrics values at every epoch. 

In [ ]:
                            # input data
seq_history = seq_model.fit(x = x_train, 
                            # input labels
                            y = y_train, 
                            # epoch -- an iteration over the entire x and y dataset provided
                            epochs = 2, 
                            # data on which to evaluate the loss and any model metrics at the ennd of each epoch
                            validation_data = (x_valid, y_valid))

Generally, fitting the "best" model may take a number of iterations -- possibly needing several hyperparameter adjustments! Once complete, the final step is to evaluate the model using the *evaluate()* method on the test set. Recall that the model is evaluated only ONCE on the test set. 

In [ ]:
# returns loss value & metrics values for model in test mode (default batchsize is 32)
test_loss, test_accuracy = seq_model.evaluate(x_test, y_test)
print(f'train loss: {test_loss:.3f}')
print(f'train accuracy: {100 * test_accuracy:.3f}%')

In summary, the general steps to constructing a model using the Sequential API are: <br> 
1. Sequential instantiation, with a list of layers
2. Compiling the model, indicating the desired loss function, optimizer, and metric(s)
3. Fitting the model with the dataset
4. Evaluating the model on the test set

While the Sequential API is easy to use, we cannot create models that share layers, have branches, nor have multiple inputs/outputs. However, we *can* with the Functional API. 

#### TO DO: Part 1 Questions
You may consult Tensorflow documentation and online sources for any of the questions below. <br>
a) Describe the architecture of *seq_model* above. (ie. how many/what types of layers? what is a dense? flatten?) <br>

*your answer here*

b) What happens if you do not specify an activation function for any one of the dense layers? <br>

*your answer here*

c) The three dense layers in *seq_model* have 200960, 32896, and 1290 trainable parameters respectively. Based on your knowledge of densely connected neural networks, explain how these numbers are derived. <br> 

*your answer here*

d) Why is the loss function sparse categorical crossentropy rather than just categorical crossentropy? <br>

*your answer here*

e) What happens if you do not specify a number of epochs in *fit()*? <br>

*your answer here*

## 2. Functional API - Basics

*func_model* below contains the same architecture as *seq_model*, but uses the Functional API. Carefully examine the code below and read the comments. Compare it with the Sequential syntax above. What differences and similarities do you notice? 

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model 

# define input tensor
input_layer = Input(shape = (28, 28, 1))

# stack layers using the syntax: current_layer()(previous_layer)
conv1 = Conv2D(16, (3, 3), activation="relu")(input_layer)
mp1 = MaxPooling2D(2, 2)(conv1)
conv2 = Conv2D(32, (3, 3), activation="relu")(mp1)
mp2 = MaxPooling2D(2, 2)(conv2)
flattened = Flatten()(mp2)
fc1 = Dense(256, activation = "relu")(flattened)
fc2 = Dense(128, activation = "relu")(fc1)
predictions = Dense(10, activation = "softmax")(fc2)

# define model object -- specify input and outputs
func_model = Model(inputs = [input_layer], outputs = [predictions])

# displays model layers (+ layer (type), output shape, param #)
func_model.summary()

As with the Sequential model, we call the *compile()* method and specify a loss function, optimizer, and metrics. Then, call the *fit()* method.  

In [ ]:
func_model.compile(loss = "sparse_categorical_crossentropy", 
                   optimizer = "sgd", 
                   metrics = ["accuracy"])

func_history = func_model.fit(x_train, 
                              y_train, 
                              epochs = 2, 
                              validation_data = (x_valid, y_valid))

Finally, we evaluate the model on the test set.

In [ ]:
# returns loss value & metrics values for model in test mode (default batchsize is 32)
test_loss, test_accuracy = func_model.evaluate(x_test, y_test)
print(f'train loss: {test_loss:.3f}')
print(f'train accuracy: {100 * test_accuracy:.3f}%')

In summary, the general steps to constructing a model using the Functional API are: <br>
1. explicitly defining the input layer
2. defining model layers, connecting each layer using Python functional syntax
3. defining the model by calling the model object and giving it the input and output layers
4. compiling --> fitting (several iterations) --> evaluating


#### TO DO: Part 2 Questions 
You may consult Tensorflow documentation and online sources for the questions below. <br>
a) In the context of modeling, what are the advantage(s) of using the functional syntax? <br>

*your answer here*

b) Notice that when defining the model object, the parameter names are plural (inputs vs input, outputs vs output). Why is this the case? <br>

*your answer here*

c) Refit either *seq_model* or *func_model*, but with epochs = 15. Then evaluate the corresponding metrics in the following cell and plot two double-line graphs. You should color code your lines and import any necessary libraries: <br>
1. train & validation loss for every epoch 
2. train & validation accuracy for every epoch  

Based on your graph(s), at how many epochs would you stop training? ( in section 3, you will work with callbacks for early stopping once a desired metric value is reached!) <br>

*your answer here*

In [ ]:
# part c -- select one
seq_metrics = seq_history.history
func_metrics = func_history.history

print(type(seq_metrics), type(func_metrics))

""" ### your code below ### """

## 3. Callbacks

When fitting your model with a specified number of epochs, you can automate the prevention of further training when certain conditions are met -- ex. when accuracy or loss reaches a certain threshold, when accuracy hasn't improved after a specified number of epochs, etc. 

The training loop supports *callbacks* such that after every epoch, there is a callback to a code function that evaluates your metrics and decides whether to continue or stop training. 

Carefully examine the code and associated comments below. 

In [ ]:
# imports abstract base class used to build new callbacks
from tensorflow.keras.callbacks import Callback

class myCallback(Callback): 
    # on_epoch_end -- called whenever an epoch ends
    # sends a log object that contains information about the current training state
    def on_epoch_end(self, epoch, logs = {}): 
        # queries the training accuracy metric, checking if it is greater than or equal to 0.93
        if (logs.get('accuracy') >= 0.93): 
            print("\nAt least 93% training accuracy reached! Training stopped.")
            self.model.stop_training = True
            
# class instantiation 
callbacks = myCallback()

# creates new Sequential model 
def create_model(): 
    model = Sequential([
        Flatten(input_shape=(28, 28)), 
        Dense(256, activation="relu"), 
        Dense(10, activation="softmax")
    ])
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="sgd", 
                  metrics=["accuracy"])
    return model

that_model = create_model()
that_model.fit(x_train, 
               y_train, 
               epochs = 10,
               # use the callbacks parameter to pass the instance above in a list
               callbacks = [callbacks]);

#### TO DO: Callbacks Exercise

Refit *func_model* such that the classifier trains to a training loss of 0.2 or below -- training should stop once training loss is at or below 0.2. 

Print "0.2 training loss reached! Training stopped" when desired loss is reached. 

In [ ]:
class myCallback(### your code here ##3): 
    def on_epoch_end(self, epochs, logs = {}): 
        ### your code here ###
    
callbacks = ### your code here ###

this_model = create_model()    
this_.fit(x_train, y_train, ### your code here###);

#### To Do: Callbacks Debugging

The code below attempts to stop training once 97% validation accuracy is reached. However, there seems to be a TypeError. (TypeError: '>=' not supported between instances of 'NoneType' and 'float') Figure out why this is the case.

In [ ]:
# do not modify myCallback class
class myCallback(Callback): 
    def on_epoch_end(self, epoch, logs = {}): 
        if (logs.get('val_accuracy') >= 0.97): 
            print("\nValidation Accuracy of 97% reached. Training Stopped.")
            self.model.stop_training = True
            
callbacks = myCallback()
fixme_model = create_model()
fixme_model.fit(x_train, y_train, epochs=20, callbacks = [callbacks])

## 4. Functional API -- Inception Network

As mentioned in section 2, the Functional API allows one to create layers that are impossible with the Sequential API -- ex. parallel layers, concatenations, multiple inputs/ouputs etc. This section explores such layers with functional syntax, and focuses on implementing the following Inception Network.

Why an Inception Network? <br>
The salient parts of can image can vary greatly in size. For example, an image labeled "dog" may have the dog either occupying most of the image, a part of the image, or very little of the image. This makes choosing the right kernel size challenging because smaller kernels are preferred for locally distributed information and larger kernels are preferred for more globally distributed information. <br>
The Inception Network allows different filter sizes on the same level, and the results of each filter are concatenated into the next layer. This also makes the network "wider" rather than "deeper", hence less computationally expensive and prone to overfitting. 

<img src="inception.png">

An arrow leaving layer *x* and pointing towards layer *y* means that layer *y* follows layer *x*. Notice that several arrows leaving from the same layer indicates **branching** and several arrows pointing to the same layer indicates a **concatenation**. Do you notice any repetition in the graph above? The same inception module is applied twice!

#### TO DO: Coding the Inception Module
The two code cells below implement the inception network in the image above -- fill in *pass* with the correct code. Your clues are all in the image above and code below -- use your knowledge of convolutional neural networks!! There should be 513,120 total trainable parameters. 

In [ ]:
from tensorflow.keras.layers import concatenate

def inception_mod(input_layer, br1, br2_in, br2_out, br3_in, br3_out, br4_out):
    # 3x3 convolution
    conv3 = Conv2D(pass, (1, 1), padding="same", activation="relu")(pass)
    conv3 = Conv2D(pass, (3, 3), padding="same", activation="relu")(pass)
    # 5x5 convolution 
    conv5 = Conv2D(pass, (1, 1), padding="same", activation="relu")(pass)
    conv5 = Conv2D(pass, (pass, pass), padding = "same", activation="relu")(conv5)
    # 3x3 max pool
    pool3 = MaxPooling2D((3, 3), strides=(1, 1), padding="same")(pass)
    pool3 = Conv2D(pass, (1, 1), padding="same", activation="relu")(pass)
    # 1x1 convolution
    conv1 = Conv2D(pass, (pass, pass), padding="same", activation="relu")(input_layer)
    # concatenation
    return concatenate([pass, pass, pass, pass], axis = -1)

In [ ]:
input_layer = Input(shape=(256, 256, 3))

module1 = inception_mod(pass, 64, 96, 128, 16, 32, 32)
module2 = inception_mod(module1, 128, 128, 192, 32, 96, 64)

model = Model(inputs = pass, outputs = pass)
model.summary()